In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

import random
import shutil
import pathlib
os.chdir(#root data directory )


In [ ]:
path = #root data directory path
path = pathlib.Path(path)
breeds = list(path.glob('*/*.jpg'))
print(len(breeds))

test_dir = #refer to data preprocessing script to set this path
train_dir = #refer to data preprocessing script to set this path

In [ ]:
os.mkdir("/kaggle/working/iv3")
import os
import random
import requests
import cv2
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model

from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [ ]:
train_gen = ImageDataGenerator(
        rescale = 1./255,
        featurewise_center = False,  
        samplewise_center = False, 
        featurewise_std_normalization = False,  
        samplewise_std_normalization = False,  
        zca_whitening = False,  
        rotation_range = 10,  
        zoom_range = 0.1, 
        width_shift_range = 0.2,  
        height_shift_range = 0.2, 
        horizontal_flip = True,  
        vertical_flip = False) 

test_gen = ImageDataGenerator(
        rescale = 1./255)

In [ ]:
train_generator = train_gen.flow_from_directory(
        train_dir,
        target_size = (299, 299),
        batch_size = 32,
        )
test_generator = test_gen.flow_from_directory(
        test_dir,
        target_size = (299, 299),
        batch_size = 32)

In [ ]:
train_steps = int(15435/32)
test_steps = int(5145/32)

In [ ]:
base_model = InceptionV3(weights = 'imagenet', include_top = False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation = 'relu')(x)
x = Dense(512, activation = 'relu')(x)
predictions = Dense(120, activation = 'softmax')(x)

model = Model(inputs = base_model.input, outputs = predictions)

In [ ]:
for layer in base_model.layers:
    layer.trainable = False
    
model.compile(optimizer = 'rmsprop', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
filepath =#set path
checkpoint = ModelCheckpoint(filepath, 
                             monitor='val_accuracy', 
                             verbose=1, 
                             save_best_only=True, 
                             mode='max', 
                             save_weights_only=False)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                                   patience=1, verbose=1, mode='min',
                                   min_delta=0.0001, cooldown=2, min_lr=1e-7)

early_stop = EarlyStopping(monitor="val_loss", mode="min", patience=10)

In [ ]:
def show_final_history(history):
    fig, ax = plt.subplots(1, 2, figsize=(15,5))
    ax[0].set_title('loss')
    ax[0].plot(history.epoch, history.history["loss"], label="Train loss")
    ax[0].plot(history.epoch, history.history["val_loss"], label="Validation loss")
    ax[1].set_title('acc')
    ax[1].plot(history.epoch, history.history["accuracy"], label="Train acc")
    ax[1].plot(history.epoch, history.history["val_accuracy"], label="Validation acc")
    ax[0].legend()
    ax[1].legend()


In [ ]:
history = model.fit_generator(train_generator, 
                    steps_per_epoch = train_steps,
                    validation_data = test_generator,
                    validation_steps = test_steps,
                    epochs = 32,
                    verbose = 1,
                    callbacks = [checkpoint, reduce_lr, early_stop])

In [ ]:
show_final_history(history)

In [ ]:
from tensorflow.keras.models import load_model
model = load_model(filepath)

for layer in model.layers[:249]:
    layer.trainable = False
for layer in model.layers[249:]:
    layer.trainable = True

from tensorflow.keras.optimizers import SGD
model.compile(optimizer = SGD(lr = 0.0001, momentum = 0.9), loss = 'categorical_crossentropy', metrics = ['accuracy'])

filepath = #set path
checkpoint = ModelCheckpoint(filepath, 
                             monitor='val_accuracy', 
                             verbose=1, 
                             save_best_only=True, 
                             mode='max', 
                             save_weights_only=False)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                                   patience=1, verbose=1, mode='min',
                                   min_delta=0.0001, cooldown=2, min_lr=1e-7)

early_stop = EarlyStopping(monitor="val_loss", mode="min", patience=10)

history = model.fit_generator(train_generator, 
                    steps_per_epoch = train_steps, 
                    epochs = 20,
                    validation_data = validation_generator,
                    validation_steps = validate_steps,
                    verbose = 1,
                    callbacks = [checkpoint, reduce_lr, early_stop])

show_final_history(history)